In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install xlrd
!pip install openpyxl 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings(action='ignore') 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,ExtraTreesClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train=pd.read_excel('/kaggle/input/predicting-food-delivery-time/Participants Data/Data_Train.xlsx')
test=pd.read_excel('/kaggle/input/predicting-food-delivery-time/Participants Data/Data_Test.xlsx')
sample=pd.read_excel('/kaggle/input/predicting-food-delivery-time/Participants Data/Sample_Submission.xlsx')


In [ ]:
# Train data preprocessing(Referring to Sabin Hashmi code for data preprocessing, I would like to thank Sabin Hashmi.)
train['Restaurant']=le.fit_transform(train['Restaurant'])
train['Minimum_Order']=pd.to_numeric(train['Minimum_Order'].str.replace('₹',' '))
train['Average_Cost']=pd.to_numeric(train['Average_Cost'].str.replace('[^0-9]',''))
train['Rating']=pd.to_numeric(train['Rating'].apply(lambda x : np.nan if x in ['Temporarily Closed','Opening Soon','-','NEW'] else x))
train['Votes']=pd.to_numeric(train['Votes'].apply(lambda x : np.nan if x=='-' else x))
train['Reviews']=pd.to_numeric(train['Reviews'].apply(lambda x : np.nan if x=='-' else x))
train['Delivery_Time']=pd.to_numeric(train['Delivery_Time'].str.replace('[^0-9]',''))

In [ ]:
# Test data preprocessing
test['Restaurant']=le.fit_transform(test['Restaurant'])
test['Minimum_Order']=pd.to_numeric(test['Minimum_Order'].str.replace('₹',' '))
test['Average_Cost']=pd.to_numeric(test['Average_Cost'].str.replace('[^0-9]',''))
test['Rating']=pd.to_numeric(test['Rating'].apply(lambda x : np.nan if x in ['Temporarily Closed','Opening Soon','-','NEW'] else x))
test['Votes']=pd.to_numeric(test['Votes'].apply(lambda x : np.nan if x=='-' else x))
test['Reviews']=pd.to_numeric(test['Reviews'].apply(lambda x : np.nan if x=='-' else x))
test['Rating']=test['Rating'].fillna(test['Rating'].median())
test['Votes']=test['Votes'].fillna(test['Votes'].mode()[0])
test['Reviews']=test['Reviews'].fillna(test['Reviews'].median())
test['Average_Cost']=test['Average_Cost'].fillna(test['Average_Cost'].mean())
sample['Delivery_Time'] = sample['Delivery_Time'].str.replace(' minutes','').astype('int64')

In [ ]:
# train location data one hot encoding
for i in range(len(train)):
    train['Location'][i] = train['Location'][i].split(',')
    train['Location'][i] = [line.strip() for line in train['Location'][i]]

train_location_df = pd.DataFrame()

for n in range(len(train)):
    location = train['Location'][n]
    location_df = pd.DataFrame()
    for i in location:
        location_df[i] = [1]
        
    train_location_df = pd.concat([train_location_df, location_df], ignore_index=True)

train_location_df.rename(columns ={"" : 'None'}, inplace = True )
train_location_df.head()

In [ ]:
# train cuisines data one hot encoding
for i in range(len(train)):
    train['Cuisines'][i] = train['Cuisines'][i].split(',')
    train['Cuisines'][i] = [line.strip() for line in train['Cuisines'][i]]

train_cuisines_df = pd.DataFrame()

for n in range(len(train)):
    cuisines = train['Cuisines'][n]
    cuisines_df = pd.DataFrame()
    for i in cuisines:
        cuisines_df[i] = [1]
        
    train_cuisines_df = pd.concat([train_cuisines_df, cuisines_df], ignore_index=True)

train_cuisines_df.rename(columns ={"" : 'None'}, inplace = True )
train_cuisines_df.head()

In [ ]:
# test location data one hot encoding
for i in range(len(test)):
    test['Location'][i] = test['Location'][i].split(',')
    test['Location'][i] = [line.strip() for line in test['Location'][i]]

test_location_df = pd.DataFrame()

for n in range(len(test)):
    location = test['Location'][n]
    location_df = pd.DataFrame()
    for i in location:
        location_df[i] = [1]
        
    test_location_df = pd.concat([test_location_df, location_df], ignore_index=True)

test_location_df.rename(columns ={"" : 'None'}, inplace = True )
test_location_df.head()

In [ ]:
# test cuisines data one hot encoding

for i in range(len(test)):
    test['Cuisines'][i] = test['Cuisines'][i].split(',')
    test['Cuisines'][i] = [line.strip() for line in test['Cuisines'][i]]
test_cuisines_df = pd.DataFrame()

for n in range(len(test)):
    cuisines = test['Cuisines'][n]
    cuisines_df = pd.DataFrame()
    for i in cuisines:
        cuisines_df[i] = [1]
        
    test_cuisines_df = pd.concat([test_cuisines_df, cuisines_df], ignore_index=True)
    
test_cuisines_df.rename(columns ={"" : 'None'}, inplace = True )
test_cuisines_df.head()   

In [ ]:
# Fill columns without each other
number = 0
ls=[]
for i in train_cuisines_df.columns:
    ls.append(i)
    ls[number].replace(' ','')
    number = number+1
    
number = 0
ls2=[]
for i in test_cuisines_df.columns:
    ls2.append(i)
    ls2[number].replace(' ','')
    number = number +1
    
complement = list(set(ls) - set(ls2))
complement2 = list(set(ls2) - set(ls))

for i in complement2:
    train_cuisines_df[i] = np.nan
train_cuisines_df.rename(columns ={"" : 'None'}, inplace = True )
train_cuisines_df.head()   

for i in complement:
    test_cuisines_df[i] = np.nan
test_cuisines_df.rename(columns ={"" : 'None'}, inplace = True )
test_cuisines_df.head()

In [ ]:
# Normalize and fill in missing values
train_02=train.copy()
del train_02['Cuisines']
del train_02['Location']
train_02=train_02.drop('Delivery_Time',axis=1)
scaler=MinMaxScaler()
X_scaled_d = scaler.fit_transform(train_02)
X_scaled = pd.DataFrame(X_scaled_d, index=train_02.index, columns=train_02.columns)
X_scaled = X_scaled[['Restaurant']]
train_02 = pd.merge(X_scaled,train_location_df, left_index =True, right_index =True, how= 'left')
train_02 = pd.merge(train_02,train_cuisines_df, left_index =True, right_index =True, how= 'left')
train_02 = train_02.fillna(0)
x=train_02
y=train['Delivery_Time']

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=50)

#  LogisticRegression 

In [ ]:
model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)
scores = cross_val_score(model_lr, X_train,y_train, cv=10) # model, train, target, cross validation
print('cross-val-score \n{}'.format(scores))
print('cross-val-score.mean \n{:.3f}'.format(scores.mean()))

In [ ]:
pred = model_lr.predict(X_test)
print("Test Accuracy: {}%".format(round(model_lr.score(X_test, y_test)*100, 2)))
print(classification_report(y_test, pred))

In [ ]:
model_coef = pd.DataFrame(data=model_lr.coef_[0][:20], index=x.columns[:20], columns=['Model Coefficient'])
model_coef.sort_values(by='Model Coefficient', ascending=False, inplace=True)
plt.bar(model_coef.index, model_coef['Model Coefficient'])
plt.xticks(rotation=90)
plt.grid()
plt.show()

# XGBClassifier

In [ ]:
model_xgb = XGBClassifier()
train_X = X_train.values
model_xgb.fit(train_X, y_train)
scores = cross_val_score(model_xgb, train_X,y_train, cv=10) # model, train, target, cross validation
print('cross-val-score \n{}'.format(scores))
print('cross-val-score.mean \n{:.3f}'.format(scores.mean()))

In [ ]:
test_X = X_test.values
pred = model_xgb.predict(test_X)
print("Test Accuracy: {}%".format(round(model_xgb.score(test_X, y_test)*100, 2)))

print(classification_report(y_test, pred))

In [ ]:
fig = plt.figure(figsize=(10, 10))

model_coef = pd.DataFrame(data=model_xgb.feature_importances_, index=x.columns, columns=['Model Coefficient'])
model_coef.sort_values(by='Model Coefficient', ascending=False, inplace=True)
plt.barh(model_coef.index[:30], model_coef['Model Coefficient'][:30])
plt.xticks(rotation=90)
plt.grid()
plt.show()

# DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
scores = cross_val_score(dt, X_train,y_train, cv=10) # model, train, target, cross validation
print('cross-val-score \n{}'.format(scores))
print('cross-val-score.mean \n{:.3f}'.format(scores.mean()))

In [ ]:
pred = dt.predict(X_test)
print("Test Accuracy: {}%".format(round(dt.score(X_test, y_test)*100, 2)))
print(classification_report(y_test, pred))

In [ ]:
fig = plt.figure(figsize=(10, 10))

model_coef = pd.DataFrame(data=dt.feature_importances_, index=x.columns, columns=['Model Coefficient'])
model_coef.sort_values(by='Model Coefficient', ascending=False, inplace=True)
plt.barh(model_coef.index[:30], model_coef['Model Coefficient'][:30])
plt.xticks(rotation=90)
plt.grid()
plt.show()

# KNN

In [ ]:
best_Kvalue = 0
best_score = 0
for i in range(2,15):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    pred = knn.predict(X_test)
    score = round(knn.score(X_test, y_test)*100,2)
#     print(f"Test Accuracy: {score}%, knn number {i}")
    if score > best_score:
        
        best_score = round(score)
        best_Kvalue = i
        
print(f"Best KNN Value: {best_Kvalue}")
print(f"Test Accuracy: {best_score}%")

In [ ]:
knn = KNeighborsClassifier(n_neighbors=best_Kvalue)
knn.fit(X_train, y_train)
pred = knn.predict(X_test)
print("KNN Classifier report: \n\n", classification_report(y_test, pred))

# SVM

In [ ]:
svm = SVC(random_state=42, gamma="auto")
svm.fit(X_train, y_train)
scores = cross_val_score(svm, X_train,y_train, cv=10) 
print('cross-val-score \n{}'.format(scores))
print('cross-val-score.mean \n{:.3f}'.format(scores.mean()))


In [ ]:
pred = svm.predict(X_test)
print("Test Accuracy: {}%".format(round(svm.score(X_test, y_test)*100, 2)))
print("SVM Classifier report: \n\n", classification_report(y_test, pred))

# RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
scores = cross_val_score(rf, X_train,y_train, cv=10) 
print('cross-val-score \n{}'.format(scores))
print('cross-val-score.mean \n{:.3f}'.format(scores.mean()))


In [ ]:
ypred = rf.predict(X_test)
print("Test Accuracy: {}%".format(round(rf.score(X_test, y_test)*100, 2)))
print("Random Forest Classifier report: \n\n", classification_report(y_test, ypred))

In [ ]:
fig = plt.figure(figsize=(10, 10))

model_coef = pd.DataFrame(data=rf.feature_importances_, index=x.columns, columns=['Model Coefficient'])
model_coef.sort_values(by='Model Coefficient', ascending=False, inplace=True)
plt.barh(model_coef.index[:30], model_coef['Model Coefficient'][:30])
plt.xticks(rotation=90)
plt.grid()
plt.show()

# AdaBoostClassifier

In [ ]:
ada=AdaBoostClassifier()
ada.fit(X_train, y_train)
scores = cross_val_score(ada, X_train,y_train, cv=10) 
print('cross-val-score \n{}'.format(scores))
print('cross-val-score.mean \n{:.3f}'.format(scores.mean()))

In [ ]:
ypred = ada.predict(X_test)
print("Test Accuracy: {}%".format(round(ada.score(X_test, y_test)*100, 2)))
print("Random Forest Classifier report: \n\n", classification_report(y_test, ypred))

In [ ]:
fig = plt.figure(figsize=(10, 10))

model_coef = pd.DataFrame(data=ada.feature_importances_, index=x.columns, columns=['Model Coefficient'])
model_coef.sort_values(by='Model Coefficient', ascending=False, inplace=True)
plt.barh(model_coef.index[:30], model_coef['Model Coefficient'][:30])
plt.xticks(rotation=90)
plt.grid()
plt.show()

# BaggingClassifier

In [ ]:
bag=BaggingClassifier()
bag.fit(X_train, y_train)
scores = cross_val_score(bag, X_train,y_train, cv=10) 
print('cross-val-score \n{}'.format(scores))
print('cross-val-score.mean \n{:.3f}'.format(scores.mean()))


In [ ]:
ypred = bag.predict(X_test)
print("Test Accuracy: {}%".format(round(bag.score(X_test, y_test)*100, 2)))
print("Random Forest Classifier report: \n\n", classification_report(y_test, ypred))

# ExtraTreesClassifier

In [ ]:
xtree=ExtraTreesClassifier()
xtree.fit(X_train, y_train)
scores = cross_val_score(xtree, X_train,y_train, cv=10) 
print('cross-val-score \n{}'.format(scores))
print('cross-val-score.mean \n{:.3f}'.format(scores.mean()))


In [ ]:
ypred = xtree.predict(X_test)
print("Test Accuracy: {}%".format(round(xtree.score(X_test, y_test)*100, 2)))
print("Random Forest Classifier report: \n\n", classification_report(y_test, ypred))

In [ ]:
fig = plt.figure(figsize=(10, 10))

model_coef = pd.DataFrame(data=xtree.feature_importances_, index=x.columns, columns=['Model Coefficient'])
model_coef.sort_values(by='Model Coefficient', ascending=False, inplace=True)
plt.barh(model_coef.index[:30], model_coef['Model Coefficient'][:30])
plt.xticks(rotation=90)
plt.grid()
plt.show()

# TEST

In [ ]:
test_02=test.copy()
del test_02['Cuisines']
del test_02['Location']
scaler=MinMaxScaler()
X_scaled_d = scaler.fit_transform(test_02)
X_scaled = pd.DataFrame(X_scaled_d, index=test_02.index, columns=test_02.columns)
X_scaled = X_scaled[['Restaurant']]
test_02 = pd.merge(X_scaled,test_location_df, left_index =True, right_index =True, how= 'left')
test_02 = pd.merge(test_02,test_cuisines_df, left_index =True, right_index =True, how= 'left')

test_02 = test_02.fillna(0)

x=test_02


In [ ]:
test_predict = rf.predict(x)
test_pd =pd.DataFrame(rf.predict(x))
print(test_pd[0].value_counts())
print(accuracy_score(sample['Delivery_Time'],test_predict))


In [ ]:
#xgb test
x_values = x.values
test_predict = model_xgb.predict(x_values)
test_pd =pd.DataFrame(model_xgb.predict(x_values))
print(test_pd[0].value_counts())
print(accuracy_score(sample['Delivery_Time'],test_predict))